### Import packages

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import pydub # For audio processing

### Ingest data

### EDA

### Modelling

In [ ]:
# Load the model.
model = hub.load('https://tfhub.dev/google/yamnet/1')

#### Model inference / Create Embeddings

#### Model classification layer

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(1024,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # z. B. 10 Klassen
])